In [6]:
!pip install transformers networkx matplotlib torch

  Using cached torch-2.4.0-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.4.0-cp311-none-macosx_11_0_arm64.whl (62.1 MB)
Using cached sympy-1.13.2-py3-none-any.whl (6.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [7]:
import networkx as nx
import matplotlib.pyplot as plt
from transformers import pipeline

In [9]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.